In [2]:
%matplotlib inline
import numpy as np
import scipy as sp
import scipy.sparse as spar
import scipy.special as spec
import sys

In [3]:
V = 1000 # nr words in vocabulary
M = 100 # nr documents
K = 20 # nr of topics
alpha = .1 # dirichlet hyperparameter

X = np.random.binomial(1,.1, size=M*V).reshape(M,V)
X = spar.csr_matrix(X, dtype=float)

In [ ]:
# For even a reasonable setup like 10K vocabulary, 5K documents and 20 topics, the size of the tensor indexed by
# <document, word, topic> simply explodes to 7.5G. This is why we can't explicitly keep all of $\phi$ in the memory.
# Instead, we iterate over the documents one by one, and accumulate the phi parameter

In [12]:
nr_terms = X.sum(axis=1) 
nr_terms = np.array(nr_terms).squeeze()

In [30]:
# model parameters
beta = np.zeros((K, V))

# variational and temp variables
gamma = np.zeros((K, M)) + alpha + (nr_terms/float(K)) # mth document, i th topic
beta_acc = np.ones((K, V))

for epoch in range(100):
    # E-step
    for m in range(M): # iterate over all documents
        phi = np.zeros((K, V), dtype=float) + 1./K

        ixw = (X[m, :] > 0).toarray().squeeze() # an index to words which have appeared in the document
        gammad = gamma[:, m] # slice for the document only once

        for ctr in range(int(1e4)): 
            # cache the values
            phi_prev = phi
            gammad_prev = gammad

            # update phi
            # WARN: exp digamma underflows < 1e-3! 
            phi[:, ixw] = ((beta[:, ixw]).T * np.exp(spec.digamma(gammad))).T 
            phi = phi / np.sum(phi, 0) # normalize phi columns

            # update gamma
            gammad = alpha + np.sum(phi, axis=1)

            # check for convergence
            dphinorm = np.linalg.norm(phi - phi_prev, "fro")
            dgammadnorm = np.linalg.norm(gammad - gammad_prev)
            
            if dphinorm < .01 and dgammadnorm < .01:
                break
            else:
                print (dphinorm, dgammadnorm)

        gamma[:, m] = gammad
        beta_acc += phi * ixw

    # M-step
    # TODO: check for numerical stability
    beta = (beta_acc.T / sum(beta_acc, 1)).T # normalize beta rows

(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:24: RuntimeWarning: invalid value encountered in divide


KeyboardInterrupt: 

In [16]:
beta.shape

(20, 1000)

In [20]:
beta * ixw.toarray()

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [28]:
beta

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])